In [1]:
!pip install -q langchain langchain-groq langchain-core pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
api_key = userdata.get('GROQ_API_KEY')

In [18]:
from langchain_groq import ChatGroq
chat = ChatGroq(api_key = api_key, model = "llama-3.3-70b-versatile", temperature = 0.2)

In [4]:
chat.invoke("hi") #testing the model instantiation

AIMessage(content="It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 36, 'total_tokens': 59, 'completion_time': 0.083636364, 'prompt_time': 0.004618614, 'queue_time': 0.224620243, 'total_time': 0.088254978}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_5d5eebccf4', 'finish_reason': 'stop', 'logprobs': None}, id='run-51e5c79e-c35e-4c8f-9b8b-70cea1be2c2c-0', usage_metadata={'input_tokens': 36, 'output_tokens': 23, 'total_tokens': 59})

In [5]:
from typing import TypedDict, List
from typing_extensions import Annotated

class AssessmentOutput(TypedDict):
    """Assessment output containing individual KPIs (as percentages out of 100), recommended packages, and a summary report."""
    stress_management: Annotated[float, "Percentage score (0-100) for Stress Management KPI. Reflects how well the user manages daily stress."]
    motivation: Annotated[float, "Percentage score (0-100) for Motivation KPI. Represents the user's drive and energy to pursue wellness goals."]
    restless_night_score: Annotated[float, "Percentage score (0-100) for Restless Night Score KPI. Indicates the frequency of sleep disturbances and overall sleep quality."]
    anxiety_level: Annotated[float, "Percentage score (0-100) for Anxiety Level KPI. Measures how often the user experiences anxiety, especially before sleep."]
    burnout_level: Annotated[float, "Percentage score (0-100) for Burnout Level KPI. Reflects how overwhelmed or emotionally drained the user feels."]
    physical_fitness_score: Annotated[float, "Percentage score (0-100) for Physical Fitness KPI. Assesses the user's level of physical activity and strength."]
    dietary_habit_score: Annotated[float, "Percentage score (0-100) for Diet & Nutrition KPI. Evaluates the user's satisfaction with and balance of their eating habits."]
    focus_score: Annotated[float, "Percentage score (0-100) for Cognitive Performance and Focus KPI. Measures the user's ability to concentrate and stay focused on tasks."]
    overall_wellness_score: Annotated[float, "Aggregated overall wellness percentage score (0-100) based on all the individual KPIs."]
    package: Annotated[List[str], "List of recommended packages. Options include 'Focus', 'Fitness', and 'Insomnia'. The recommendation can be a single package, a combination of packages, or all three packages based on the assessment."]
    report: Annotated[str, "A detailed summary report of the assessment results and recommendations based on the KPI analysis."]


In [6]:
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import PromptTemplate

In [12]:
template = """
You are an AI wellness assessment system designed to evaluate users' well-being and provide personalized insights. This assessment system analyzes responses and generates wellness reports, recommending specific services and packages from *DailyWellnessAI* to help users improve their mental and physical health. While this system itself is not called *DailyWellnessAI*, all recommendations are part of the *DailyWellnessAI* platform.

Based on the user's responses to the following 15 questions, predict the following KPIs as percentages out of 100:
- stress_management
- motivation
- restless_night_score
- anxiety_level
- burnout_level
- physical_fitness_score
- dietary_habit_score
- focus_score
- overall_wellness_score

**Recommendation System:**
Based on the predicted KPIs, recommend one of the following *DailyWellnessAI* packages:
- **Focus** (for low concentration, motivation, or burnout)
- **Insomnia** (for sleep disturbances, restlessness, or anxiety before sleep)
- **Fitness** (for low physical activity, poor dietary habits, or weak physical strength)

If the **overall_wellness_score** is greater than 90 (or 95), recommend **all three packages** and let the user choose among them, emphasizing that *DailyWellnessAI* offers holistic well-being solutions.

**Summary Report:**
Provide a brief wellness summary explaining the predicted KPIs and recommended package(s) based on the user's responses. Justify the recommendations by highlighting key factors affecting the user's well-being and how *DailyWellnessAI* can help improve them.

### Assessment Questions:

1. How often do you feel stressed by daily tasks?
   Options: Never, Sometimes, Often, Always
   Answer: {ans1}

2. How well do you handle stress?
   Options: Excellent, Good, Fair, Poor
   Answer: {ans2}

3. How would you rate your current level of burnout?
   Options: None, Mild, Moderate, Severe
   Answer: {ans3}

4. How well can you concentrate on your daily tasks?
   Options: Excellent, Good, Fair, Poor
   Answer: {ans4}

5. How often do you have trouble falling asleep?
   Options: Never, Rarely, Sometimes, Always
   Answer: {ans5}

6. How would you rate the quality of your sleep?
   Options: Excellent, Good, Fair, Poor
   Answer: {ans6}

7. How often do you wake up during the night?
   Options: Never, Rarely, Sometimes, Often
   Answer: {ans7}

8. How often do you feel anxious before sleep?
   Options: Never, Rarely, Sometimes, Often
   Answer: {ans8}

9. How happy are you with your eating habits?
   Options: Very happy, Happy, Unhappy, Very unhappy
   Answer: {ans9}

10. How balanced is your diet?
    Options: Very balanced, Balanced, Unbalanced, Very unbalanced
    Answer: {ans10}

11. How often do you exercise or do physical activity?
    Options: Daily, Several times a week, Once a week, Never
    Answer: {ans11}

12. How would you rate your physical strength?
    Options: Excellent, Good, Fair, Poor
    Answer: {ans12}

13. How motivated are you to work on your wellness goals daily?
    Options: Very motivated, Moderately motivated, Slightly motivated, Not at all
    Answer: {ans13}

14. How would you rate your overall health and well-being?
    Options: Excellent, Good, Fair, Poor
    Answer: {ans14}

15. Any more remarks about yourself that you want to add?
    Answer: {ans15}

Use the responses to generate KPI predictions, determine the recommended *DailyWellnessAI* package(s), and provide a concise summary report that briefly justifies the recommendations.
"""

prompt_template = PromptTemplate(
    input_variables=[
        "ans1", "ans2", "ans3", "ans4", "ans5", "ans6", "ans7", "ans8", "ans9",
        "ans10", "ans11", "ans12", "ans13", "ans14", "ans15"
    ],
    template=template
)


In [13]:
sample_inputs = {
    "ans1": "Always",
    "ans2": "Poor",
    "ans3": "Severe",
    "ans4": "Poor",
    "ans5": "Always",
    "ans6": "Poor",
    "ans7": "Often",
    "ans8": "Often",
    "ans9": "Unhappy",
    "ans10": "Unbalanced",
    "ans11": "Never",
    "ans12": "Poor",
    "ans13": "Not at all",
    "ans14": "Poor",
    "ans15": "I constantly feel fatigued, have trouble sleeping, and struggle to focus. My stress levels are high, and I feel physically weak and unmotivated."
}

formatted_prompt = prompt_template.format(**sample_inputs)
print(formatted_prompt)


You are an AI wellness assessment system designed to evaluate users' well-being and provide personalized insights. This assessment system analyzes responses and generates wellness reports, recommending specific services and packages from *DailyWellnessAI* to help users improve their mental and physical health. While this system itself is not called *DailyWellnessAI*, all recommendations are part of the *DailyWellnessAI* platform.

Based on the user's responses to the following 15 questions, predict the following KPIs as percentages out of 100:
- stress_management
- motivation
- restless_night_score
- anxiety_level
- burnout_level
- physical_fitness_score
- dietary_habit_score
- focus_score
- overall_wellness_score

**Recommendation System:**  
Based on the predicted KPIs, recommend one of the following *DailyWellnessAI* packages:  
- **Focus** (for low concentration, motivation, or burnout)  
- **Insomnia** (for sleep disturbances, restlessness, or anxiety before sleep)  
- **Fitness*

In [21]:
sample_inputs_2 = {
    "ans1": "Sometimes",
    "ans2": "Fair",
    "ans3": "Mild",
    "ans4": "Poor",
    "ans5": "Rarely",
    "ans6": "Good",
    "ans7": "Rarely",
    "ans8": "Sometimes",
    "ans9": "Happy",
    "ans10": "Balanced",
    "ans11": "Several times a week",
    "ans12": "Good",
    "ans13": "Moderately motivated",
    "ans14": "Good",
    "ans15": "I struggle with maintaining focus and often get distracted easily. However, my sleep quality is decent, and I try to stay active through regular workouts."
}

formatted_prompt_2 = prompt_template.format(**sample_inputs_2)
print(formatted_prompt)



You are an AI wellness assessment system designed to evaluate users' well-being and provide personalized insights. This assessment system analyzes responses and generates wellness reports, recommending specific services and packages from *DailyWellnessAI* to help users improve their mental and physical health. While this system itself is not called *DailyWellnessAI*, all recommendations are part of the *DailyWellnessAI* platform.

Based on the user's responses to the following 15 questions, predict the following KPIs as percentages out of 100:
- stress_management
- motivation
- restless_night_score
- anxiety_level
- burnout_level
- physical_fitness_score
- dietary_habit_score
- focus_score
- overall_wellness_score

**Recommendation System:**  
Based on the predicted KPIs, recommend one of the following *DailyWellnessAI* packages:  
- **Focus** (for low concentration, motivation, or burnout)  
- **Insomnia** (for sleep disturbances, restlessness, or anxiety before sleep)  
- **Fitness*

In [14]:
structured_llm = chat.with_structured_output(AssessmentOutput)

In [24]:
response_2 = structured_llm.invoke(formatted_prompt_2)

In [25]:
response_2

{'anxiety_level': 60,
 'burnout_level': 40,
 'dietary_habit_score': 80,
 'focus_score': 40,
 'motivation': 60,
 'overall_wellness_score': 65,
 'package': ['Focus'],
 'physical_fitness_score': 70,
 'report': "The user experiences moderate anxiety and burnout levels, with notable difficulties in maintaining focus. Their dietary habits are balanced, and they engage in regular physical activity. However, they struggle with concentration and motivation. The recommended 'Focus' package from DailyWellnessAI can help improve their focus, motivation, and overall well-being by providing tailored strategies and support.",
 'restless_night_score': 20,
 'stress_management': 50}

int